<a href="https://colab.research.google.com/github/Aditi1203/CMPE-295/blob/master/2DCNN_Butterworth_CYBHi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Aditi1203/CMPE-295.git

Cloning into 'CMPE-295'...
remote: Enumerating objects: 20886, done.
remote: Counting objects: 100% (20886/20886), done.
remote: Compressing objects: 100% (20883/20883), done.
remote: Total 1173018 (delta 9), reused 20870 (delta 2), pack-reused 1152132
Receiving objects: 100% (1173018/1173018), 3.44 GiB | 30.75 MiB/s, done.
Resolving deltas: 100% (220409/220409), done.
Checking out files: 100% (758793/758793), done.


In [2]:
pwd

'/content'

In [3]:
cd /content/CMPE-295/Dataset_CYBHi/scalogram_module/

/content/CMPE-295/Dataset_CYBHi/scalogram_module


In [4]:
import pandas as pd
import numpy as np
import cv2
import os
import imutils
from keras.applications import VGG19, VGG16
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg16 import preprocess_input
from PIL import Image
import keras
import tensorflow as tf
from keras.models import Model, load_model
from keras import Sequential, losses, optimizers, Input
from keras.layers import Dense, Conv2D, MaxPool2D, Average, Flatten, Dropout, Activation
from keras.utils import to_categorical, plot_model,vis_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard, EarlyStopping
from keras.preprocessing.image import img_to_array
from skimage import io
import imutils
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pywt
from scipy import signal
from scipy.spatial import distance


Using TensorFlow backend.


In [0]:
class DataModel:
    """
            DataModel
            not used currently in the program.
    """
    def __init__(self):
        self.items_labels = None
        self.images = []
        self.curated_data = {}
    def add_image(self, image_array):
        self.images.append(image_array)
    def add_labels(self, df):
        self.item_labels = df
    def add_curated_data(self, image, emotion):
        self.curated_data['segment'] = image #Array from csv of each segment
        self.curated_data['label'] = label #Label
        self.curated_data['scalogram'] =scaledImage #scalogram of each segment

def load_data(number_of_items=90):
    """
        number_of_items -> Number of items to return
        returns the data in a dictionary of images and labels.
    """
    path = "Scalogram"
    path1= "Scalogram/Cropped"
    data = [] 
    curated_data = {"label":[], "scalogram":[]}
    for subject_name in os.listdir(path)[:number_of_items]:
        # At the start of the iteration build a data model
        data_model = DataModel()
        if subject_name == ".DS_Store":
            continue
        if subject_name  ==".ipynb_checkpoints":
            continue
        print ("Going through subject:" + subject_name)
        base=os.path.basename(path+"/"+subject_name)
        labelData=os.path.splitext(base)[0]
        print(labelData)
        i=0
        for items in os.listdir(path+"/"+subject_name):
            if items == ".DS_Store":
                continue
            if items.endswith(".png"):
                #i=i+1
                #print(i)
                try:
                    im2 = cv2.imread(path+"/"+subject_name+"/"+items)
                    #plt.imshow(im2)
                    #plt.show()
                    crop_img = im2[30:20+235, 50:50+342]
                    im = cv2.resize(crop_img, (224,224)) # Changing into 80x80X3
                    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
                    #print(type(crop_img))
                    #plt.imshow(crop_img)
                    #plt.show()
                    

              #curated_data['segments'].append(df1)
                    #curated_data['scalogram'].append(crop_img)
                    curated_data['scalogram'].append(im)
                    curated_data['label'].append(labelData)
              #print(df1)
                except:
                      df = None  
        #data.append(data_model) # Save all the data

    return curated_data


In [6]:
data = load_data()

Going through subject:person_JCC
person_JCC
Going through subject:person_MGA
person_MGA
Going through subject:person_MP
person_MP
Going through subject:person_JN
person_JN
Going through subject:person_RF
person_RF
Going through subject:person_JPA
person_JPA
Going through subject:person_DB
person_DB
Going through subject:person_FM
person_FM
Going through subject:person_JCA
person_JCA
Going through subject:person_ARA
person_ARA
Going through subject:person_TV
person_TV
Going through subject:person_MB
person_MB
Going through subject:person_RD
person_RD
Going through subject:person_JS
person_JS
Going through subject:person_AG
person_AG
Going through subject:person_SF
person_SF
Going through subject:person_FO
person_FO
Going through subject:person_HF
person_HF
Going through subject:person_JA
person_JA
Going through subject:person_DC
person_DC
Going through subject:person_CB
person_CB
Going through subject:person_MQ
person_MQ
Going through subject:person_MC
person_MC
Going through subject:pe

In [7]:
temp=np.array(data['scalogram'])
print(temp.shape)

(10367, 224, 224)


In [8]:
encoder = LabelEncoder()
encoder.fit(data['label'])

LabelEncoder()

In [0]:
data['label'] = encoder.transform(data['label'])

In [0]:
all_the_classes = encoder.classes_
mapping = {
    0: 'person_JCC',
    1: 'person_MGA',
    2: 'person_MP',
    3: 'person_JN',
    4: 'person_RF',
    5: 'person_JPA',
    6: 'person_DB',
    7: 'person_FM',
    8: 'person_JCA',
    9: 'person_ARA',
    10: 'person_TV',
    11: 'person_MB',
    12: 'person_RD',
    13: 'person_JS',
    14: 'person_AG',
    15: 'person_SF',
    16: 'person_HF',
    17: 'person_JA',
    18: 'person_DC',
    19: 'person_CB',
    20: 'person_MQ',
    21: 'person_MC',
    22: 'person_ACA',
    23: 'person_JSA',
    24: 'person_RA',
    25: 'person_MBA',
    26: 'person_MJR',
    27: 'person_ABD',
    28: 'person_JL',
    29: 'person_IC',
    30: 'person_CSR',
    31: 'person_FP',
    33: 'person_ARL',
    32: 'person_JM',
    34: 'person_RR',
    35: 'person_RL',
    36: 'person_TF',
    37: 'person_AFS',
    38: 'person_MMJ',
    39: 'person_AL',
    40: 'person_GF',
    41: 'person_ARF',
    42: 'person_PES',
    43: 'person_RRA',
    44: 'person_RAA',
    45: 'person_IB',
    46: 'person_JP',
    47: 'person_AR',
    48: 'person_JB',
    49: 'person_CF',
    50: 'person_MA',
    51: 'person_DS',
    52: 'person_SR',
    53: 'person_TC',
    54: 'person_VM',
    55: 'person_VO',
    56: 'person_AC',
    57: 'person_AA',
    58: 'person_PM',
    59: 'person_PMA',
    60: 'person_JV',
    61: 'person_JC',
    62: 'person_AA'
} 


#print(mapping)


# Get emotion from class number   
def get_name_from_class(class_number):
    """
        gets the corresponding subject from the class
    """
    if mapping.get(class_number,None):
        return mapping.get(class_number)
    else:
        return -1 # No such class

In [0]:
# Extracting features and labels
features = np.array(data['scalogram'])
labels = data['label']

In [21]:
print ("Shape of features: ", features.shape)
print ("Shape of labels: ", labels.shape)

Shape of features:  (10367, 224, 224)
Shape of labels:  (10367,)


In [0]:
# Changing
labels = to_categorical(labels, num_classes=len(mapping))

In [0]:
# Building model
model = Sequential()
model.add(Conv2D(64, kernel_size=5, input_shape=(224, 224, 1), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=5, activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
#model.add(Dense(150, activation="relu"))
model.add(Dense(120, activation="relu"))
model.add(Dense(100, activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(80, activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(len(mapping), activation="softmax"))

In [24]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 220, 220, 64)      1664      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 106, 106, 64)      102464    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 53, 53, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 53, 53, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 179776)           

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizers.RMSprop(),metrics=['accuracy'])

In [0]:
# Splitting
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=1)

In [0]:
features_train, features_val, labels_train, labels_val = train_test_split(features_train, labels_train, test_size=0.25, random_state=1)

In [0]:
tensorboard = TensorBoard()
earlystopping = EarlyStopping(patience=3)

In [0]:
# Scaling
features_train = features_train.astype("float32")

In [0]:
features_test = features_test.astype("float32")

In [0]:
features_val = features_val.astype("float32")

In [0]:
features_train = features_train / 1/255

In [0]:
features_test = features_test / 1/255

In [0]:
features_val = features_val / 1/255

In [0]:
features_train = features_train.reshape(len(features_train), 224, 224, 1)

In [0]:
features_test = features_test.reshape(len(features_test), 224, 224, 1)

In [0]:
features_val = features_val.reshape(len(features_val), 224, 224, 1)

In [39]:
# Training
model.fit(features_train, labels_train, epochs=100, batch_size=32, callbacks=[tensorboard,earlystopping], validation_data=(features_test, labels_test))

Train on 6219 samples, validate on 2074 samples
Epoch 1/100
6219/6219 [==============================] - 56s 9ms/step - loss: 4.3527 - accuracy: 0.0310 - val_loss: 3.8336 - val_accuracy: 0.0593
Epoch 2/100
6219/6219 [==============================] - 48s 8ms/step - loss: 2.9566 - accuracy: 0.2352 - val_loss: 2.5271 - val_accuracy: 0.2604
Epoch 3/100
6219/6219 [==============================] - 48s 8ms/step - loss: 2.1021 - accuracy: 0.3843 - val_loss: 1.9921 - val_accuracy: 0.4315
Epoch 4/100
6219/6219 [==============================] - 48s 8ms/step - loss: 1.7634 - accuracy: 0.4666 - val_loss: 2.4514 - val_accuracy: 0.3515
Epoch 5/100
6219/6219 [==============================] - 48s 8ms/step - loss: 1.5775 - accuracy: 0.5149 - val_loss: 1.8703 - val_accuracy: 0.4619
Epoch 6/100
6219/6219 [==============================] - 47s 8ms/step - loss: 1.4501 - accuracy: 0.5478 - val_loss: 1.7112 - val_accuracy: 0.5106
Epoch 7/100
6219/6219 [==============================] - 48s 8ms/step - loss

In [0]:
model.save("2DCNN_ButterworthKalmann_CYBHi.h5") # Saving the model

In [41]:
pwd

'/content/CMPE-295/Dataset_CYBHi/scalogram_module'

In [0]:
# create authentication model with base model
base_model = keras.models.load_model('2DCNN_ButterworthKalmann_CYBHi.h5')
base_model.layers.pop()
authenticate_model = Model(base_model.input, base_model.layers[-1].output)
for layer in authenticate_model.layers: 
  layer.trainable = False

# k_seg = []
# k_seg_prediction= []
# person_label = get_emotion_from_class(np.argmax(labels_val[300]))
   
# taking an element from K_seg to act as current segment
# person_107 = k_seg[0]
# k_seg = k_seg[1:]
# print(person_107) 
# auth_prediction = authenticate_model.predict(person_107.reshape(1,224,224,1))[0]


def dissimilarity_calc(person):
  # print(current_vector)
  # print(current_vector.shape)
  k_seg_vector = prediction_dict[person]
  dists = distance.cdist(k_seg_vector, k_seg_vector, 'euclidean')
  
  max_dist = []
  for i in range(len(dists)):
    mean=np.mean(dists[i])
    standard_deviation=np.std(dists[i])
    dist_mean=abs(dists[i]-mean)
    soln_array=dist_mean<(standard_deviation)
    soln_array1=dists[i][soln_array]
    max_dist.append(max(soln_array1))

  average = sum(max_dist)/len(max_dist)
  print("threshold for ",person," is:",average)
  return(average)



In [0]:
def authenticate_user(username, path_to_scalogram, threshold = 0.0003):
  username_vector_list = []
  for i in range(len(features_train)):
    if get_emotion_from_class(np.argmax(labels_train[i])) == username:
      username_vector_list.append(authenticate_model.predict(features_train[i].reshape(1,224,224,1))[0])
  #get image data
  # print(username_vector_list)
  im = cv2.imread(path_to_scalogram)
  crop_img = im[30:20+235, 50:50+342]
  im = cv2.resize(crop_img, (224,224)) # Changing into 80x80X3
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im = im.astype('float32')
  im = im / 1/255
  scalogram_vector = authenticate_model.predict(im.reshape(1,224,224,1))[0]
  # print(scalogram_vector)
  dissimilarity_score = dissimilarity_calc(scalogram_vector, username_vector_list)
  # print(dissimilarity_score)
  if dissimilarity_score <= threshold:
    print ("Authenticated with dissimilarity score: ", dissimilarity_score)
  else:
    print ("Not Authenticated due to high dissimilarity: ", dissimilarity_score)




In [44]:
#Segregate segments based on the subject label
def create_map():
  map = {}
  for i in range(len(features_train)):
    print(i)
    if get_emotion_from_class(np.argmax(labels_train[i])) in map.keys():
       map[get_emotion_from_class(np.argmax(labels_train[i]))].append(authenticate_model.predict(features_train[i].reshape(1,224,224,1))[0])
    else:
      map[get_emotion_from_class(np.argmax(labels_train[i]))] = []
      map[get_emotion_from_class(np.argmax(labels_train[i]))].append(authenticate_model.predict(features_train[i].reshape(1,224,224,1))[0])
  return map
   
prediction_dict = create_map()
# print(map['person_100'])

Streaming output truncated to the last 5000 lines.
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393
1394
1395
1396
1397
1398
1399
1400
1401
1402
1403
1404
1405
1406
1407
1408
1409

In [45]:
# cell for calculating average threshold

threshold_dict = {}
for person in prediction_dict:
  threshold_dict[person] = dissimilarity_calc(person)


threshold for  person_AC  is: 0.34846115276855844
threshold for  person_ABD  is: 0.8745420146859911
threshold for  person_JB  is: 0.7961234591923196
threshold for  person_AA  is: 1.13816413048423
threshold for  person_ARF  is: 0.8340715464348109
threshold for  person_DB  is: 0.9479949963568716
threshold for  person_JCA  is: 1.0200912381177303
threshold for  person_CB  is: 0.9262567599008603
threshold for  person_TF  is: 0.3947297282556689
threshold for  person_CSR  is: 0.8839527187837046
threshold for  person_ARL  is: 0.6773476513353756
threshold for  person_JN  is: 0.8601719310963709
threshold for  person_MQ  is: 0.45708537483177536
threshold for  person_MBA  is: 0.8011704902160407
threshold for  person_MB  is: 0.8329402267309532
threshold for  person_RA  is: 0.2196964768127366
threshold for  person_JC  is: 0.48535135243603406
threshold for  person_JV  is: 0.21202198644505382
threshold for  person_PES  is: 0.5253566868166247
threshold for  person_IB  is: 0.5659492989376391
threshold f

In [0]:
def similar_score(test_template, stored_templates):
  dists = distance.cdist(test_template, stored_templates, 'euclidean')
  soln=min(dists[0])
  return soln

In [47]:
tp = 0
tn = 0
fp = 0
fn = 0

for i in range(len(features_test)):
  true_label = get_emotion_from_class(np.argmax(labels_test[i]))
  current_segment = authenticate_model.predict(features_test[i].reshape(1,224,224,1))[0]
  for person in prediction_dict:
    test_distance = similar_score(current_segment.reshape(1,current_segment.shape[0]),prediction_dict[person])
    if test_distance <= threshold_dict[person]:
      if person == true_label:
        tp += 1
      else:
        fp += 1
    else:
      if person == true_label:
        fn += 1
      else:
        tn += 1
  print(true_label)
print('tp = ',tp)
print('tn = ',tn)
print('fp = ',fp)
print('fn = ',fn)
accuracy = ((tp+tn)/(tp+fp+tn+fn))*100
print('accuracy:',accuracy)

false_acceptance = ((fp)/(fp+fn))
print('false_acceptance:',false_acceptance)

false_rejection = fn/(tp+tn)
print('false_rejection:',false_rejection)




person_JS
person_MQ
person_JC
person_DB
person_CF
person_TF
person_DC
person_MMJ
person_MP
person_AL
person_JN
person_JM
person_HF
person_DS
person_JM
person_MBA
person_MBA
person_MMJ
person_IC
person_VO
person_MB
person_RRA
person_AG
person_AFS
person_VO
person_JN
person_JL
person_JA
person_JC
person_RAA
person_JCC
person_DC
person_MBA
person_DS
person_AG
person_JA
person_VO
person_FM
person_ACA
person_ARF
person_ARF
person_AA
person_SR
person_JC
person_MJR
person_RAA
person_DC
person_JCC
person_RA
person_AA
person_VM
person_MBA
person_TV
person_MBA
person_JV
person_MMJ
person_ARL
person_AC
person_PM
person_JS
person_MMJ
person_TC
person_IB
person_TV
person_PMA
person_MGA
person_FP
person_RD
person_FM
person_DB
person_MC
person_MBA
person_JSA
person_DS
person_JN
person_VM
person_RF
person_MA
person_RA
person_RR
person_JCC
person_RF
person_VO
person_AA
person_ACA
person_MBA
person_ABD
person_JSA
person_JC
person_RR
person_MP
person_HF
person_JM
person_PM
person_ARA
person_DC
person_RF


In [0]:
subject = "person_200"
claim = "person_201"

# finding prediction vector of the first segment of the subject's test data
for i in range(1,len(features_test)):
  if get_emotion_from_class(np.argmax(labels_test[i])) == subject:
    current_segment_prediction = authenticate_model.predict(features_test[i].reshape(1,224,224,1))[0]
    break
score=similar_score(current_segment_prediction.reshape(1,48), map[claim])
print("threshold for ",claim,":",threshold_dict[claim])
if score>threshold_dict[claim]:
  print("You are not authenticated with dissimarity score:{}".format(score))
else:
  print("Authentication Successful! with dissimarity score:{}".format(score))

threshold for  person_201 : 0.470854108331471
You are not authenticated with dissimarity score:1.1754314537884056


In [0]:
#Map to store all the throshold values per subject
threshold_map = {}

In [0]:
def calc_accuracy():
  for i in range(len(features_train)):
    subject_name = get_emotion_from_class(np.argmax(labels_train[i]))
    subject_vector = authenticate_model.predict(features_train[i].reshape(1,224,224,1))[0]
    username_vector_list = map[subject_name]
    username_vector_list = authenticate_model.predict(np.array(username_vector_list))
    dissimilarity_score = dissimilarity_calc(subject_vector, username_vector_list)
    if subject_name in threshold_map.keys():
      threshold_map[subject_name] = threshold_map[subject_name] + dissimilarity_score
    else:
      threshold_map[subject_name] = dissimilarity_score
    print(i, subject_name, dissimilarity_score)
  

In [0]:
 for j in threshold_map.keys():
    threshold_map[j] = threshold_map[j]/len(map[j])

In [0]:
authenticate_user('person_101', 'Scalogram/person_100/seg20.csv.png')

Not Authenticated due to high dissimilarity:  0.0012213722425328595


In [0]:
# Test an image
def test_image(image,label):
    #print(image.shape)
    prediction = np.argmax(model.predict(image.reshape(1,224,224,1)))
    result_predict=get_emotion_from_class(prediction)
    actual=np.argmax(label)
    result_actual=get_emotion_from_class(actual)
    #print(prediction)
    #print(actual)
    return result_actual,result_predict

    
# Get emotion from class number   
def get_emotion_from_class(class_number):
    """
        gets the corresponding label from the class
    """
    if mapping.get(class_number,None):
      return mapping.get(class_number)
    else:
      return -1 # No such class

In [0]:
countFP=0
for i in range(len(features_val)):
    actual,predicted=test_image(features_val[i],labels_val[i])
    print("Model predicts: "+predicted+" and actual label is: "+actual)
    if(predicted!=actual):
        print("Wrong prediction")
        countFP=countFP+1
    print("-------------------------------------------------------------")
  
print("Total False Positives: "+str(countFP))
print("Total Positives: "+str(len(features_val)))